# Análise Exploratória de Dados – Preços de Computadores

Este trabalho consiste numa análise exploratória do dataset **“All Computer Prices”** (Kaggle), que reúne informações sobre computadores (desktop e outros), respetivos componentes e preços. O objetivo principal é compreender como diferentes características de hardware (CPU, RAM, armazenamento, GPU, potência da fonte, etc.) se relacionam com o preço final das máquinas.

Ao longo deste notebook serão desenvolvidas as seguintes etapas:

- **Descrição inicial do dataset**: dimensão, tipos de variáveis e breve caracterização das principais colunas.
- **Limpeza e preparação dos dados**: identificação e tratamento de valores em falta, verificação de linhas duplicadas, correção de tipos de dados e remoção de variáveis irrelevantes.
- **Transformação dos dados**: criação e/ou recodificação de variáveis que facilitem a análise (por exemplo, categorias de componentes ou faixas de preço).
- **Análise exploratória univariada e multivariada**: estudo da distribuição das variáveis individuais e das relações entre componentes e preço (correlações, comparações entre grupos, etc.).
- **Dados agrupados e tabelas resumidas**: utilização de `groupby`, `pivot_table` e tabelas cruzadas para analisar médias, totais e contagens por categoria (por exemplo, por tipo de componente ou gama de computador).
- **Visualização de dados**: aplicação de diferentes tipos de gráficos (histogramas, boxplots, gráficos de barras, dispersão, entre outros) para apoiar a interpretação dos resultados.
- **Síntese e conclusões**: resumo dos principais padrões observados no dataset e possíveis interpretações sobre a influência dos componentes no preço dos computadores.

Num segundo notebook será ainda desenvolvido um **dashboard interativo** com recurso à livraria `panel`, permitindo explorar os dados de forma dinâmica (por exemplo, filtrando por características específicas e visualizando a variação de preços).


## 1. Carregamento das bibliotecas e do dataset

Nesta primeira etapa são carregadas as bibliotecas necessárias para a análise e é importado o ficheiro com os dados. Em seguida, é criada uma cópia do dataset original e visualizadas as primeiras linhas da tabela, de forma a confirmar que o ficheiro foi lido corretamente e a ter um primeiro contacto com a estrutura dos dados.


In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# para ver gráficos dentro do notebook
%matplotlib inline

# carregar o ficheiro (troca o nome pelo teu)
df = pd.read_csv("dados.csv")  # se for .xlsx, usa pd.read_excel("dados.xlsx")

df_original = df.copy()  # cópia do original

#carrega as 5 primeiras linhas por defeito
df.head()


,device_type,brand,model,release_year,os,form_factor,cpu_brand,cpu_model,cpu_tier,cpu_cores,...,resolution,refresh_hz,battery_wh,charger_watts,psu_watts,wifi,bluetooth,weight_kg,warranty_months,price
0,Desktop,Samsung,Samsung Forge XDI,2022,Windows,ATX,Intel,Intel i5-11129,3,12,...,2560x1440,90,0,0,750,Wi-Fi 6,5.1,11.00,36,1383.99
1,Laptop,Samsung,Samsung Pro KM8,2022,Windows,Mainstream,Intel,Intel i7-11114,4,12,...,1920x1080,90,56,120,0,Wi-Fi 6,5.3,2.03,12,2274.99
2,Desktop,Lenovo,Lenovo Strix BIE,2024,macOS,SFF,AMD,AMD Ryzen 5 5168,2,8,...,3440x1440,120,0,0,850,Wi-Fi 6,5.0,7.00,24,1879.99
3,Desktop,Dell,Dell Cube AXR,2024,Windows,ATX,AMD,AMD Ryzen 5 7550,2,6,...,3440x1440,120,0,0,650,Wi-Fi 6,5.2,6.00,36,1331.99
4,Laptop,Gigabyte,Gigabyte Pro IX1,2024,Linux,Gaming,AMD,AMD Ryzen 7 6230,5,16,...,2560x1600,90,80,90,0,Wi-Fi 6,5.2,1.50,12,2681.99


## 1.1. Análise inicial e preparação para a limpeza dos dados

Antes de iniciar o processo de limpeza e transformação dos dados, é importante compreender a estrutura geral do dataset. Nesta secção é feita uma análise inicial das dimensões da tabela, dos tipos de dados, da existência de valores em falta e das variáveis disponíveis. Esta análise exploratória preliminar permite identificar potenciais problemas (como colunas com muitos valores em falta ou tipos de dados inadequados) e preparar de forma mais informada os passos seguintes de limpeza e preparação dos dados.


In [64]:
df.shape      # nº de linhas e colunas

(100000, 33)

O comando `df.shape` devolve uma tupla com o número de linhas e o número de colunas do dataset. Este resultado permite ter uma primeira noção da dimensão do ficheiro (quantos registos existem e quantas variáveis estão disponíveis para análise).


In [65]:
df.info()         # tipos de dados, valores em falta

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 33 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   device_type          100000 non-null  object 
 1   brand                100000 non-null  object 
 2   model                100000 non-null  object 
 3   release_year         100000 non-null  int64  
 4   os                   100000 non-null  object 
 5   form_factor          100000 non-null  object 
 6   cpu_brand            100000 non-null  object 
 7   cpu_model            100000 non-null  object 
 8   cpu_tier             100000 non-null  int64  
 9   cpu_cores            100000 non-null  int64  
 10  cpu_threads          100000 non-null  int64  
 11  cpu_base_ghz         100000 non-null  float64
 12  cpu_boost_ghz        100000 non-null  float64
 13  gpu_brand            100000 non-null  object 
 14  gpu_model            100000 non-null  object 
 15  gpu_tier          

O comando `df.info()` apresenta um resumo da estrutura do DataFrame, incluindo o número de registos, o nome de cada coluna, o tipo de dados associado (por exemplo, `int64`, `float64`, `object`) e o número de valores não nulos em cada variável. Esta informação é útil para identificar desde cedo a presença de valores em falta e possíveis problemas de tipagem (por exemplo, números guardados como texto), que terão de ser tratados na fase de limpeza.


In [66]:
df.describe()     # estatísticas das colunas numéricas

,release_year,cpu_tier,cpu_cores,cpu_threads,cpu_base_ghz,cpu_boost_ghz,gpu_tier,vram_gb,ram_gb,storage_gb,storage_drive_count,display_size_in,refresh_hz,battery_wh,charger_watts,psu_watts,bluetooth,weight_kg,warranty_months,price
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.00000,100000.000000
mean,2022.320850,3.153490,10.515740,19.372700,2.591322,3.531310,2.991350,6.152180,39.706400,903.936000,1.524980,20.126655,98.464860,41.813470,61.383450,272.520500,5.084764,4.289699,22.20036,1928.764220
std,2.025761,1.373175,5.044092,9.718426,0.336435,0.350024,1.459643,3.964926,31.902684,774.243654,0.797284,6.709577,43.301652,35.868841,62.795034,354.686355,0.245977,3.814628,10.23190,580.492689
min,2018.000000,1.000000,4.000000,4.000000,2.000000,2.800000,1.000000,0.000000,8.000000,256.000000,1.000000,13.300000,60.000000,0.000000,0.000000,0.000000,4.200000,0.920000,12.00000,372.990000
25%,2021.000000,2.000000,6.000000,12.000000,2.400000,3.300000,2.000000,4.000000,16.000000,512.000000,1.000000,14.000000,60.000000,0.000000,0.000000,0.000000,5.000000,1.500000,12.00000,1503.990000
50%,2023.000000,3.000000,8.000000,16.000000,2.600000,3.500000,3.000000,6.000000,32.000000,512.000000,1.000000,16.000000,90.000000,56.000000,65.000000,0.000000,5.100000,2.000000,24.00000,1863.990000
75%,2024.000000,4.000000,14.000000,24.000000,2.800000,3.800000,4.000000,8.000000,64.000000,1024.000000,2.000000,27.000000,120.000000,70.000000,90.000000,650.000000,5.200000,7.000000,24.00000,2287.990000
max,2025.000000,6.000000,28.000000,56.000000,3.400000,4.500000,6.000000,16.000000,144.000000,4096.000000,4.000000,34.000000,240.000000,99.000000,240.000000,1200.000000,5.300000,16.000000,48.00000,10984.990000


O comando `df.describe()` calcula estatísticas descritivas básicas para as variáveis numéricas do dataset, como a média, desvio padrão, valores mínimo e máximo, e os quartis (25%, 50% e 75%). Estas estatísticas permitem ter uma visão geral da distribuição dos dados numéricos, identificar ordens de grandeza e possíveis valores extremos (outliers) que poderão influenciar as análises posteriores.


In [67]:

df.columns        # nomes das colunas


Index(['device_type', 'brand', 'model', 'release_year', 'os', 'form_factor',
       'cpu_brand', 'cpu_model', 'cpu_tier', 'cpu_cores', 'cpu_threads',
       'cpu_base_ghz', 'cpu_boost_ghz', 'gpu_brand', 'gpu_model', 'gpu_tier',
       'vram_gb', 'ram_gb', 'storage_type', 'storage_gb',
       'storage_drive_count', 'display_type', 'display_size_in', 'resolution',
       'refresh_hz', 'battery_wh', 'charger_watts', 'psu_watts', 'wifi',
       'bluetooth', 'weight_kg', 'warranty_months', 'price'],
      dtype='object')

O comando `df.columns` devolve a lista com os nomes de todas as colunas do DataFrame. A visualização destes nomes ajuda a perceber que tipo de informação está disponível (por exemplo, especificações de hardware, preço, características adicionais) e a decidir quais as variáveis que serão mais relevantes para a análise exploratória.


## 2. Limpeza de dados

Depois de compreender a estrutura geral do dataset, é necessário verificar a qualidade dos dados disponíveis. Nesta fase é feita a identificação de valores em falta e de eventuais registos duplicados. Estes problemas podem afetar as análises estatísticas e as conclusões, pelo que é importante detetá-los e, quando necessário, aplicar estratégias de correção (remoção de linhas, substituição de valores, etc.).


In [68]:
#valores em falta por coluna em ordem descendente
df.isna().sum().sort_values(ascending=False)


device_type            0
brand                  0
model                  0
release_year           0
os                     0
form_factor            0
cpu_brand              0
cpu_model              0
cpu_tier               0
cpu_cores              0
cpu_threads            0
cpu_base_ghz           0
cpu_boost_ghz          0
gpu_brand              0
gpu_model              0
gpu_tier               0
vram_gb                0
ram_gb                 0
storage_type           0
storage_gb             0
storage_drive_count    0
display_type           0
display_size_in        0
resolution             0
refresh_hz             0
battery_wh             0
charger_watts          0
psu_watts              0
wifi                   0
bluetooth              0
weight_kg              0
warranty_months        0
price                  0
dtype: int64

O comando `df.isna().sum().sort_values(ascending=False)` calcula, para cada coluna, o número de valores em falta (`NaN`) e apresenta o resultado ordenado por ordem decrescente. Desta forma, é possível identificar rapidamente quais as variáveis mais afetadas por valores em falta e que poderão necessitar de tratamento específico (por exemplo, remoção de registos, substituição por média/mediana ou criação de categorias como "desconhecido").


In [69]:
# linhas duplicadas
df.duplicated().sum()

#Caso ouvesse repetidos ---> df = df.drop_duplicates()


np.int64(0)

O comando `df.duplicated().sum()` devolve o número de linhas duplicadas no dataset, ou seja, registos que são exatamente iguais noutros pontos da tabela. A existência de duplicados pode distorcer as análises (por exemplo, ao contar ou calcular médias), pelo que, caso fossem detetadas linhas repetidas, estas poderiam ser removidas com o comando `df = df.drop_duplicates()`. No presente caso, o resultado obtido indica se é ou não necessário aplicar essa remoção.

No entanto, verificou-se que o número de linhas duplicadas é igual a 0, pelo que não foi necessário remover qualquer registo.

In [70]:
num = df.select_dtypes(include=np.number)       # só colunas numéricas
zeros_por_coluna = num.eq(0).sum().sort_values(ascending=False)
zeros_por_coluna

psu_watts              59844
battery_wh             40156
charger_watts          40156
vram_gb                13389
cpu_threads                0
cpu_cores                  0
cpu_tier                   0
release_year               0
ram_gb                     0
cpu_base_ghz               0
cpu_boost_ghz              0
gpu_tier                   0
display_size_in            0
storage_drive_count        0
storage_gb                 0
refresh_hz                 0
bluetooth                  0
weight_kg                  0
warranty_months            0
price                      0
dtype: int64


Foi analisada a frequência de valores iguais a 0 nas colunas numéricas do dataset.  
Este passo permite identificar variáveis onde o valor 0 pode significar ausência de componente, valor desconhecido ou um caso especial (por exemplo, computadores sem bateria).  

As colunas com maior número de zeros serão analisadas com mais detalhe, para decidir se estes valores são válidos ou se devem ser tratados na fase de limpeza dos dados.


In [71]:
df["battery_wh"].value_counts().head()

battery_wh
0     40156
70    13110
60    11983
80    10659
56     9135
Name: count, dtype: int64

In [72]:
df["psu_watts"].value_counts().head()

psu_watts
0      59844
650     8831
750     7923
550     7179
850     5625
Name: count, dtype: int64

In [73]:
df.loc[df["battery_wh"] == 0].head()


,device_type,brand,model,release_year,os,form_factor,cpu_brand,cpu_model,cpu_tier,cpu_cores,...,resolution,refresh_hz,battery_wh,charger_watts,psu_watts,wifi,bluetooth,weight_kg,warranty_months,price
0,Desktop,Samsung,Samsung Forge XDI,2022,Windows,ATX,Intel,Intel i5-11129,3,12,...,2560x1440,90,0,0,750,Wi-Fi 6,5.1,11.0,36,1383.99
2,Desktop,Lenovo,Lenovo Strix BIE,2024,macOS,SFF,AMD,AMD Ryzen 5 5168,2,8,...,3440x1440,120,0,0,850,Wi-Fi 6,5.0,7.0,24,1879.99
3,Desktop,Dell,Dell Cube AXR,2024,Windows,ATX,AMD,AMD Ryzen 5 7550,2,6,...,3440x1440,120,0,0,650,Wi-Fi 6,5.2,6.0,36,1331.99
5,Desktop,MSI,MSI Think KSG,2025,Windows,ATX,Intel,Intel i7-10369,5,16,...,2560x1440,90,0,0,1000,Wi-Fi 5,5.0,9.0,36,2751.99
6,Desktop,Apple,Apple Arena R5Q,2024,Windows,ATX,Apple,Apple M2,2,6,...,2560x1440,60,0,0,850,Wi-Fi 6,5.1,9.0,24,1609.99


In [74]:
df.loc[df["battery_wh"] == 0, "device_type"].value_counts(normalize=True)


device_type
Desktop    1.0
Name: proportion, dtype: float64

In [75]:
df.loc[df["psu_watts"] == 0].head()


,device_type,brand,model,release_year,os,form_factor,cpu_brand,cpu_model,cpu_tier,cpu_cores,...,resolution,refresh_hz,battery_wh,charger_watts,psu_watts,wifi,bluetooth,weight_kg,warranty_months,price
1,Laptop,Samsung,Samsung Pro KM8,2022,Windows,Mainstream,Intel,Intel i7-11114,4,12,...,1920x1080,90,56,120,0,Wi-Fi 6,5.3,2.03,12,2274.99
4,Laptop,Gigabyte,Gigabyte Pro IX1,2024,Linux,Gaming,AMD,AMD Ryzen 7 6230,5,16,...,2560x1600,90,80,90,0,Wi-Fi 6,5.2,1.50,12,2681.99
8,Laptop,Dell,Dell Creator GIQ,2024,Windows,Mainstream,Intel,Intel i9-14473,6,26,...,2560x1600,60,80,240,0,Wi-Fi 5,5.0,1.17,48,2953.99
9,Laptop,Lenovo,Lenovo Blade MIZ,2025,Windows,Ultrabook,AMD,AMD Ryzen 3 4374,1,4,...,3840x2160,120,60,45,0,Wi-Fi 6,5.3,1.50,24,1653.99
10,Laptop,HP,HP Pro 6XS,2023,Windows,Mainstream,Intel,Intel i5-13209,2,6,...,1920x1080,165,70,90,0,Wi-Fi 6E,5.2,1.24,36,1371.99


In [76]:
# entre as linhas onde psu_watts == 0,
# quantos são desktop / laptop?
df.loc[df["psu_watts"] == 0, "device_type"].value_counts(normalize=True)


device_type
Laptop    1.0
Name: proportion, dtype: float64

### Consistência entre o tipo de equipamento e a informação de ecrã

Numa fase de verificação e preparação dos dados, foi analisada a relação entre o tipo de equipamento (`device_type`) e a variável `display_size_in`, que representa o tamanho do ecrã em polegadas. Para tal, foi criado um indicador auxiliar (`has_display_info`) que identifica os registos onde existe informação de ecrã (`display_size_in` não nulo e superior a zero), e construída uma tabela de contingência entre `device_type` e `has_display_info`.

A análise desta tabela mostrou que:
- praticamente todos os laptops têm informação de ecrã associada, o que é esperado, uma vez que o monitor faz parte do próprio equipamento;
- uma parte dos desktops também surge com `display_size_in` preenchido. 

Em vez de considerar estes casos como erro, interpretou-se que estes registos correspondem a bundles, ou seja, conjuntos em que o desktop é vendido juntamente com um monitor. Esta conclusão será explorada mais à frente, na fase de transformação dos dados, através da criação de uma variável explícita que distingue desktops isolados de bundles (PC + monitor).


In [77]:
# ver quantos registos têm informação de display
df["has_display_info"] = df["display_size_in"].notna() & (df["display_size_in"] > 0)

df["has_display_info"].value_counts()


has_display_info
True    100000
Name: count, dtype: int64

In [78]:
# relação entre tipo de equipamento e existência de display
pd.crosstab(df["device_type"], df["has_display_info"])


has_display_info,True
device_type,
Desktop,40156
Laptop,59844


In [79]:
# comando para verificar se ha algum campo vazio em display
df["has_display_info"].value_counts()
df["display_size_in"].isna().sum()
df["display_size_in"].min()


np.float64(13.3)

## **3. Transformação de Dados**

### Transformação 1 – Indicador de presença de bateria (`has_battery`)

A partir da variável `battery_wh` foi criada a nova variável booleana `has_battery`, que assume o valor `True` quando a capacidade de bateria é superior a zero (`battery_wh > 0`) e `False` quando é igual a zero. 

Esta transformação permite distinguir de forma simples equipamentos com e sem bateria, facilitando a análise comparativa entre diferentes tipos de computador (por exemplo, desktops e laptops) ao longo do estudo.


In [80]:
df["has_battery"] = df["battery_wh"] > 0

df["has_battery"].value_counts()


has_battery
True     59844
False    40156
Name: count, dtype: int64

In [81]:
df[["battery_wh", "has_battery"]].head()

,battery_wh,has_battery
0,0,False
1,56,True
2,0,False
3,0,False
4,80,True


### Transformação 2 – Idade do equipamento (`age_years`)

A partir da variável `release_year` foi criada a variável `age_years`, que representa a idade do equipamento em anos. 

Para isso, foi considerado como ano de referência o ano mais recente presente no dataset, e calculada a diferença entre esse ano e o ano de lançamento de cada computador. Desta forma, equipamentos mais recentes têm `age_years = 0`, enquanto modelos mais antigos apresentam valores mais elevados. 

Esta transformação torna mais intuitiva a análise da influência da idade do computador no preço e noutras características.


In [86]:
# Transformação 2: idade do equipamento em anos

# usar o ano mais recente do dataset como referência
ano_ref = datetime.now().year
print("Ano de referência:", ano_ref)

df["age_years"] = ano_ref - df["release_year"]

df[["release_year", "age_years"]].head()


Ano de referência: 2025


,release_year,age_years
0,2022,3
1,2022,3
2,2024,1
3,2024,1
4,2024,1


### Transformação 3 – Indicador de elevada capacidade de RAM (`is_high_ram`)

A partir da variável `ram_gb` foi criada a variável booleana `is_high_ram`, que assume o valor `True` quando o equipamento possui pelo menos 16 GB de memória RAM (`ram_gb >= 16`) e `False` caso contrário.

Esta transformação permite distinguir facilmente computadores com maior capacidade de memória, o que é útil para analisar diferenças de preço e de outras características entre equipamentos com RAM elevada e equipamentos com RAM mais modesta.


In [87]:
# Transformação 3: indicador de muita RAM

df["is_high_ram"] = df["ram_gb"] > 16

df["is_high_ram"].value_counts()


is_high_ram
True     59260
False    40740
Name: count, dtype: int64

In [88]:
df[["ram_gb", "is_high_ram"]].head(10)

,ram_gb,is_high_ram
0,16,False
1,64,True
2,8,False
3,16,False
4,96,True
5,96,True
6,8,False
7,32,True
8,128,True
9,8,False


### Transformação 4 – Indicador de computador gaming (`is_gaming`)

Para identificar computadores com características mais adequadas para jogos foi criada a variável booleana `is_gaming`. Nesta análise, considerou-se como "gaming" qualquer equipamento que cumpra simultaneamente as seguintes condições:

- placa gráfica com pelo menos 8 GB de memória de vídeo (`vram_gb >= 8`);
- memória RAM elevada (`is_high_ram = True`, ou seja, pelo menos 16 GB de RAM);
- taxa de atualização do ecrã de pelo menos 120 Hz (`refresh_hz >= 120`).

Embora seja uma definição simplificada, estes critérios refletem três aspetos importantes de um computador orientado para gaming: uma placa gráfica mais potente, uma quantidade de RAM adequada e um ecrã com maior fluidez. A variável `is_gaming` será utilizada posteriormente para comparar preços e outras características entre computadores gaming e não gaming.


In [89]:
# Transformação 4: indicador de computador gaming

df["is_gaming"] = (
    (df["vram_gb"] >= 8) &
    (df["is_high_ram"]) &
    (df["refresh_hz"] >= 120)
)

df["is_gaming"].value_counts()


is_gaming
False    83644
True     16356
Name: count, dtype: int64

In [90]:
df[["ram_gb", "is_high_ram", "vram_gb", "refresh_hz", "is_gaming"]].head(20)


,ram_gb,is_high_ram,vram_gb,refresh_hz,is_gaming
0,16,False,6,90,False
1,64,True,10,90,False
2,8,False,4,120,False
3,16,False,6,120,False
4,96,True,12,90,False
5,96,True,16,90,False
6,8,False,0,60,False
7,32,True,0,60,False
8,128,True,16,60,False
9,8,False,4,120,False


### Transformação 5 – Indicador de bundle PC + monitor (`is_bundle`)

Com base na análise efetuada na fase de preparação dos dados, concluiu-se que alguns registos de desktops apresentam também um valor na coluna `display_size_in`. Estes casos foram interpretados como bundles, isto é, conjuntos em que o computador de secretária é vendido com monitor incluído.

Para representar explicitamente esta situação foi criada a variável booleana `is_bundle`, definida da seguinte forma:

- `is_bundle = True` quando o equipamento é um `Desktop` e a variável `display_size_in` está preenchida com um valor superior a zero;
- `is_bundle = False` nos restantes casos (desktops sem monitor associado e laptops).

Esta transformação permite distinguir desktops vendidos isoladamente de desktops vendidos em conjunto com monitor, possibilitando análises específicas sobre o preço e as características destes bundles.


In [91]:
# Transformação 5: indicador se o desktop vem em bundle (PC + monitor)

df["is_bundle"] = (
    (df["device_type"] == "Desktop") &
    (df["display_size_in"].notna()) &
    (df["display_size_in"] > 0)
)

df["is_bundle"].value_counts()


is_bundle
False    59844
True     40156
Name: count, dtype: int64

In [93]:
pd.crosstab(df["device_type"], df["is_bundle"])


is_bundle,False,True
device_type,,
Desktop,0,40156
Laptop,59844,0


In [92]:
df[["device_type", "display_size_in", "is_bundle"]].head(20)


,device_type,display_size_in,is_bundle
0,Desktop,27.0,True
1,Laptop,16.0,False
2,Desktop,32.0,True
3,Desktop,27.0,True
4,Laptop,15.6,False
5,Desktop,24.0,True
6,Desktop,32.0,True
7,Desktop,27.0,True
8,Laptop,14.0,False
9,Laptop,15.6,False


### Transformação 6 : Faixas de preço (`price_band`) e índice de desempenho (`performance_score`)

A variável `price` foi convertida na variável categórica `price_band`, que agrupa os computadores em quatro faixas de preço (até 500, 501–1000, 1001–2000 e mais de 2000). Esta transformação facilita a comparação entre diferentes grupos de equipamentos e será utilizada em análises com dados agrupados (por exemplo, comparar características médias entre computadores mais baratos e mais caros).

Em paralelo, foi criado o índice simples de desempenho `performance_score`, que combina a classificação da placa gráfica (`gpu_tier`) e do processador (`cpu_tier`). Foi utilizada a fórmula:

`performance_score = gpu_tier * 2 + cpu_tier`

atribuindo um peso maior à GPU, uma vez que o desempenho em jogos e em muitas aplicações de IA depende fortemente da placa gráfica, embora o processador também tenha influência. Este índice não corresponde a uma métrica oficial, mas funciona como uma aproximação prática do desempenho global do sistema, permitindo analisar, de forma mais direta, a relação entre as faixas de preço (`price_band`) e o desempenho médio dos computadores.


In [94]:
bins = [0, 500, 1000, 2000, df["price"].max()]
labels = ["até 500", "501–1000", "1001–2000", "mais de 2000"]

df["price_band"] = pd.cut(df["price"], bins=bins, labels=labels)
df["price_band"].value_counts()


price_band
1001–2000       56799
mais de 2000    41004
501–1000         2187
até 500            10
Name: count, dtype: int64

In [95]:
df["price_band"].dtype



CategoricalDtype(categories=['até 500', '501–1000', '1001–2000', 'mais de 2000'], ordered=True, categories_dtype=object)

In [96]:
# Transformação extra: índice simples de desempenho (CPU + GPU)

df["performance_score"] = df["gpu_tier"] * 2 + df["cpu_tier"]

df[["cpu_tier", "gpu_tier", "performance_score"]].head()


,cpu_tier,gpu_tier,performance_score
0,3,2,7
1,4,4,12
2,2,1,4
3,2,2,6
4,5,5,15


### Transformação 7 – Faixas de idade do equipamento (`age_band`)

A variável `age_years`, que representa a idade do computador em anos, foi convertida na variável categórica `age_band`, agrupando os equipamentos em quatro faixas de idade:

- até 1 ano;
- 2–3 anos;
- 4–5 anos;
- mais de 5 anos.

Esta transformação permite analisar de forma mais clara a distribuição dos computadores por idade e comparar o preço e outras características entre equipamentos mais recentes e mais antigos.


In [97]:
# Transformação: faixas de idade do equipamento

bins = [-1, 1, 3, 5, 100]   # limites das idades em anos
labels = ["até 1 ano", "2–3 anos", "4–5 anos", "mais de 5 anos"]

df["age_band"] = pd.cut(df["age_years"], bins=bins, labels=labels)

df["age_band"].value_counts()


age_band
2–3 anos          34074
até 1 ano         33804
4–5 anos          20002
mais de 5 anos    12120
Name: count, dtype: int64

In [98]:
df["age_band"].dtype


CategoricalDtype(categories=['até 1 ano', '2–3 anos', '4–5 anos', 'mais de 5 anos'], ordered=True, categories_dtype=object)

## **4. Análise com dados agrupados e tabelas cruzadas**

### 4.1 GroupBy


#### Dados agrupados – preço médio por tipo de equipamento

Para analisar diferenças de preço entre tipos de equipamento, foi calculado o preço médio por `device_type`. Observou-se que os valores médios diferem entre desktops e laptops, o que indica que o tipo de equipamento tem impacto no posicionamento de preço.


In [99]:
df.groupby("device_type")["price"].mean()


device_type
Desktop    1819.545284
Laptop     2002.051360
Name: price, dtype: float64

In [100]:
preco_por_tipo = df.groupby("device_type")["price"].agg(["mean", "count"]).round(2)
preco_por_tipo


,mean,count
device_type,,
Desktop,1819.55,40156
Laptop,2002.05,59844


#### Dados agrupados – preço médio em computadores gaming

Foi também calculado o preço médio separando computadores classificados como gaming (`is_gaming = True`) dos restantes. Os resultados mostram que os equipamentos gaming apresentam, em média, um preço significativamente superior, o que é consistente com as especificações de hardware mais exigentes deste tipo de máquinas.


In [101]:
df.groupby("is_gaming")["price"].mean()



is_gaming
False    1868.307142
True     2237.939560
Name: price, dtype: float64

In [102]:
preco_gaming = df.groupby("is_gaming")["price"].agg(["mean", "count"]).round(2)
preco_gaming


,mean,count
is_gaming,,
False,1868.31,83644
True,2237.94,16356


#### Dados agrupados – desempenho por faixa de preço (`price_band`)

Utilizando as faixas de preço definidas em `price_band`, foi calculada a média do índice de desempenho `performance_score` em cada grupo. Verifica-se uma tendência clara de aumento do desempenho médio à medida que o preço aumenta, sendo a faixa "mais de 2000" a que apresenta os valores de `performance_score` mais elevados.


In [103]:
df.groupby("price_band")["performance_score"].mean()


C:\Users\admin\AppData\Local\Temp\ipykernel_6736\1998620159.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("price_band")["performance_score"].mean()


price_band
até 500          4.000000
501–1000         3.695016
1001–2000        6.864117
mais de 2000    12.574944
Name: performance_score, dtype: float64

In [104]:
perf_por_preco = df.groupby("price_band")["performance_score"].mean()
perf_por_preco


C:\Users\admin\AppData\Local\Temp\ipykernel_6736\2550216405.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  perf_por_preco = df.groupby("price_band")["performance_score"].mean()


price_band
até 500          4.000000
501–1000         3.695016
1001–2000        6.864117
mais de 2000    12.574944
Name: performance_score, dtype: float64

#### Dados agrupados – preço por faixa de idade (`age_band`)

Utilizando as faixas de idade definidas em `age_band`, foi calculado o preço médio `price` em cada grupo. Observa-se que os computadores mais recentes (faixas de idade mais baixas) tendem a apresentar preços médios superiores, enquanto os equipamentos mais antigos se concentram nas faixas com preço médio mais reduzido. Esta análise confirma a tendência esperada de desvalorização do hardware ao longo do tempo, à medida que surgem modelos mais recentes e com melhor desempenho.


In [105]:
df.groupby("age_band")["price"].mean()


C:\Users\admin\AppData\Local\Temp\ipykernel_6736\1700280888.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("age_band")["price"].mean()


age_band
até 1 ano         1985.428558
2–3 anos          1934.138412
4–5 anos          1880.496699
mais de 5 anos    1835.269620
Name: price, dtype: float64

In [106]:
preco_por_idade = df.groupby("age_band")["price"].mean().round(2)
preco_por_idade


C:\Users\admin\AppData\Local\Temp\ipykernel_6736\2972126415.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  preco_por_idade = df.groupby("age_band")["price"].mean().round(2)


age_band
até 1 ano         1985.43
2–3 anos          1934.14
4–5 anos          1880.50
mais de 5 anos    1835.27
Name: price, dtype: float64

### 4.2. Análise com pivot_table

### 4.2.1. Pivot table – preço médio por tipo de equipamento e faixa de preço

Foi construída uma `pivot_table` com o preço médio (`price`) por tipo de equipamento (`device_type`) nas linhas e faixa de preço (`price_band`) nas colunas. Esta tabela permite observar, em simultâneo, como o preço médio varia entre desktops e laptops e como estes se distribuem pelas diferentes faixas de preço.

A pivot table facilita a comparação entre categorias e ajuda a identificar, por exemplo, em que intervalos de preço os desktops e os laptops são mais frequentes ou apresentam maior valor médio.


In [107]:
tabela_preco = pd.pivot_table(
    df,
    values="price",
    index="device_type",
    columns="price_band",
    aggfunc="mean"
)


C:\Users\admin\AppData\Local\Temp\ipykernel_6736\2951962532.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  tabela_preco = pd.pivot_table(


In [108]:
tabela_preco


price_band,até 500,501–1000,1001–2000,mais de 2000
device_type,,,,
Desktop,419.59,895.327987,1552.285156,2405.875815
Laptop,464.79,912.032373,1579.899059,2523.045338


### 4.2.2. Pivot table – desempenho médio por tipo de equipamento e perfil gaming

Outra `pivot_table` foi criada para analisar o índice de desempenho médio (`performance_score`) em função do tipo de equipamento (`device_type`) e da classificação gaming (`is_gaming`). Nesta tabela, cada célula representa o desempenho médio para uma combinação de tipo de computador (desktop ou laptop) e perfil (gaming ou não gaming).

Esta abordagem permite comparar, por exemplo, se os desktops gaming apresentam `performance_score` superior aos laptops gaming, bem como avaliar a diferença de desempenho entre equipamentos gaming e não gaming dentro de cada tipo de equipamento.


In [109]:
tabela_perf = pd.pivot_table(
    df,
    values="performance_score",
    index="device_type",
    columns="is_gaming",
    aggfunc="mean"
)


In [58]:
tabela_perf

is_gaming,False,True
device_type,,
Desktop,8.903681,12.143529
Laptop,8.333300,12.043576


## **4.3 – Análise com crosstab**

### 4.3.1. Crosstab – computadores gaming por tipo de equipamento

Para analisar a distribuição de computadores gaming em cada tipo de equipamento, foi utilizada a função `crosstab` entre `device_type` e `is_gaming`. A versão normalizada por linha permite obter, para cada tipo de equipamento, a proporção de computadores gaming e não gaming.

Esta análise mostra em que medida os desktops e os laptops são utilizados para perfis mais exigentes, ajudando a perceber se os computadores gaming são mais comuns num dos tipos de equipamento.


In [110]:
pd.crosstab(df["device_type"], df["is_gaming"], normalize="index")


is_gaming,False,True
device_type,,
Desktop,0.825281,0.174719
Laptop,0.843928,0.156072


### 4.3.2. Crosstab – presença de bateria por tipo de equipamento

Foi também construída uma `crosstab` entre o tipo de equipamento (`device_type`) e o indicador de presença de bateria (`has_battery`). Esta tabela permite verificar se a variável `has_battery` está consistente com a natureza dos equipamentos.

Os resultados mostram que praticamente todos os laptops têm bateria (`has_battery = True`), enquanto nos desktops este valor é residual ou inexistente, o que confirma a coerência desta variável com a distinção entre computadores portáteis e de secretária.


In [111]:
pd.crosstab(df["device_type"], df["has_battery"], normalize="index")


has_battery,False,True
device_type,,
Desktop,1.0,0.0
Laptop,0.0,1.0


### 4.3.3. Crosstab – perfil gaming por faixa de preço (`price_band`)

Por fim, foi analisada a relação entre as faixas de preço (`price_band`) e o indicador de computador gaming (`is_gaming`), recorrendo novamente a uma `crosstab` normalizada por linha. 

Esta tabela mostra, para cada intervalo de preço, qual a proporção de computadores gaming e não gaming. Observa-se que, nas faixas de preço mais baixas, a presença de equipamentos gaming é muito reduzida, aumentando de forma significativa nas faixas de preço mais elevadas, o que é coerente com o custo acrescido das configurações orientadas para jogos.


In [112]:
pd.crosstab(df["price_band"], df["is_gaming"], normalize="index")


is_gaming,False,True
price_band,,
até 500,1.000000,0.000000
501–1000,0.995428,0.004572
1001–2000,0.904875,0.095125
mais de 2000,0.733124,0.266876
